<a href="https://colab.research.google.com/github/alina-chak/RESEARCH_PHD/blob/main/PCA_Experimentation_8_9_2022_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/titania7777/PCA-CBVR.git

Cloning into 'PCA-CBVR'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 69 (delta 19), reused 47 (delta 9), pack-reused 0
Unpacking objects: 100% (69/69), done.


In [ ]:
%cd /content/PCA-CBVR

In [2]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%cd /content/PCA-CBVR/DB/UCF101

/content/PCA-CBVR/DB/UCF101


In [4]:
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
!wget https://www.dropbox.com/s/j4brxpdcqsp848n/ucf101_labels.zip

--2022-09-08 14:20:15--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  65.4MB/s    in 2m 57s  

2022-09-08 14:23:13 (37.3 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]

--2022-09-08 14:23:14--  https://www.dropbox.com/s/j4brxpdcqsp848n/ucf101_labels.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/j4brxpdcqsp848n/ucf101_labels.zip [following]
--2022-09-08 14:23:14--  https://www.dropbox.com/s/raw/j4brxpdcqsp848n/ucf101_labels.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import shutil
shutil.copy("/content/drive/MyDrive/RESEARCH/Semantic Feature Based Video Retrieval_Objective2/ucf101_labels.zip", "/content/PCA-CBVR/DB/UCF101/ucf101_labels.zip")

'/content/PCA-CBVR/DB/UCF101/ucf101_labels.zip'

In [10]:
import shutil
shutil.copy("/content/drive/MyDrive/RESEARCH/Semantic Feature Based Video Retrieval_Objective2/UCF101.rar", "/content/PCA-CBVR/DB/UCF101/UCF101.rar")

'/content/PCA-CBVR/DB/UCF101/UCF101.rar'

In [14]:
#For removing access of drive
drive.flush_and_unmount()

In [15]:
#for permanent removal of drive directories
!rm -rf /content/drive

In [16]:
!unrar x UCF101.rar && mv UCF-101 videos && rm -rf UCF101.rar
!unzip ucf101_labels.zip && rm -rf ucf101_labels.zip

Streaming output truncated to the last 5000 lines.
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g09_c04.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c01.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c02.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c03.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c04.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c05.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c06.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g10_c07.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g11_c01.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g11_c02.avi             62%  OK 
Extracting  UCF-101/PlayingGu

In [17]:
%cd /content/PCA-CBVR

/content/PCA-CBVR


In [ ]:
!python preprocess.py

[1] video_path:./DB/UCF101/videos
[2] save_path:./DB/UCF101/frames
[3] frame_size:240
[4] num_workers:-1
12006/13320 (320x240) -> (320x240) length: 435   name: v_PommelHorse_g07_c03

In [ ]:
CATEGORIES = []
import os
dirname = os.listdir("/content/PCA-CBVR/DB/UCF101/videos")

for dir in dirname:
  print(dir)
  CATEGORIES.append(dir)


In [ ]:
#DEEP FEATURES BASED CLUSTERING BASED KEY FRAME SELECTION Too time consuming
import cv2
import numpy as np
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False
import glob
import cv2
import os

from sklearn.cluster import KMeans, spectral_clustering
from sklearn.metrics import silhouette_score, pairwise_distances_argmin_min
from scipy.signal import find_peaks

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

import tensorflow as tf
import shutil

#FUNCTIONS FOR CLUSTERING FOR KEY FRAMES

def sort_frames(list_of_frames, video_id, filename):
    return sorted(list_of_frames, key=lambda x: int(x.split(f'/content/PCA-CBVR/DB/UCF101/frames/{video_id}/{filename}/')[1].split('.jpeg')[0]))

def get_image_features(frames_filepaths):
  image_features = []
  model = VGG16(weights='imagenet', include_top=False)
  for img_path in frames_filepaths:
      print(img_path)
      img = image.load_img(img_path, target_size=(224, 224))
      img_data = image.img_to_array(img)
      img_data = np.expand_dims(img_data, axis=0)
      img_data = preprocess_input(img_data)

      vgg16_feature = model.predict(img_data)
      image_features.append(np.array(vgg16_feature).flatten())
  return image_features

def kmeans(image_features, n_clusters):
  silhouette = np.zeros(n_clusters)
  for k in range(2, n_clusters):
    kmeans = KMeans(n_clusters=k).fit_predict(image_features)
    silhouette[k] = silhouette_score(image_features, kmeans)
    print(f'{k} cluster complete')
  return silhouette

def random_frame_idxs_for_cluster(kmeans_labels):
  idxs = []
  for label in range(len(np.unique(kmeans_labels))):
    idxs.append(np.random.choice(np.argwhere(kmeans_labels == label).flatten(), 1)[0])
  return np.sort(np.array(idxs))

def closest_to_centroid_frames(kmeans, image_list, image_features):
  closest, _ = pairwise_distances_argmin_min(
      kmeans.cluster_centers_, image_features)
  return np.array(image_list)[closest]

def create_video_from_frames(frames, title, fps=30, fourcc=cv2.VideoWriter_fourcc('M','J','P','G')):
    img_array = []
    for filename in frames:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)

    out = cv2.VideoWriter(title, fourcc, 30, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()


for category in CATEGORIES:
  print("Processing category %s" % category)
  cou = 0
  # Get all files in current category's folder.
  #folder_path = os.path.join("..", "R3D50", category)
  filenames = os.listdir('/content/PCA-CBVR/DB/UCF101/frames/' + category)
  print(filenames)
  for filename in filenames:
    frames = os.listdir('/content/PCA-CBVR/DB/UCF101/frames/' + category + "/" + filename)
    ec_images = glob.glob('/content/PCA-CBVR/DB/UCF101/frames/' + category + "/" + filename + '/*')
    print(ec_images)
    ec_images = sort_frames(ec_images, category, filename)
    images_features = get_image_features(ec_images)
    kmeans5 = KMeans(n_clusters=2, random_state=42).fit(images_features)
    five_frames = closest_to_centroid_frames(kmeans5, ec_images, images_features)
    sorted_five_frames = sort_frames(five_frames, category, filename)
    original_frames = sort_frames(sorted_five_frames, category, filename)
    #print(original_frames)
    try: 
      if not os.path.exists(f'/content/PCA-CBVR/DB/UCF101/New/{category}'): 
        os.makedirs(f'/content/PCA-CBVR/DB/UCF101/New/{category}')
          
        
    except OSError: 
      print ('Error: Creating directory of data')
    create_video_from_frames(original_frames, '/content/PCA-CBVR/DB/UCF101/New/'+ category + '/' + filename +'.avi')




In [ ]:
!git clone https://github.com/soCzech/TransNetV2.git #Change the variables of deep SBD model

In [ ]:
!pip install tensorflow==2.1

In [ ]:
#2 kmeans TRANSNET SBD
import os
import numpy as np
import tensorflow as tf

__file__ = "/content/KTH-Action-Recognition/dataset/TransNetV2/inference/"
class TransNetV2:

    def __init__(self, model_dir=None):
        if model_dir is None:
            model_dir = os.path.join(os.path.dirname(__file__), "transnetv2-weights/")
            if not os.path.isdir(model_dir):
                raise FileNotFoundError(f"[TransNetV2] ERROR: {model_dir} is not a directory.")
            else:
                print(f"[TransNetV2] Using weights from {model_dir}.")

        self._input_size = (27, 48, 3)
        try:
            self._model = tf.saved_model.load(model_dir)
        except OSError as exc:
            raise IOError(f"[TransNetV2] It seems that files in {model_dir} are corrupted or missing. "
                          f"Re-download them manually and retry. For more info, see: "
                          f"https://github.com/soCzech/TransNetV2/issues/1#issuecomment-647357796") from exc

    def predict_raw(self, frames: np.ndarray):
        assert len(frames.shape) == 5 and frames.shape[2:] == self._input_size, \
            "[TransNetV2] Input shape must be [batch, frames, height, width, 3]."
        frames = tf.cast(frames, tf.float32)

        logits, dict_ = self._model(frames)
        single_frame_pred = tf.sigmoid(logits)
        all_frames_pred = tf.sigmoid(dict_["many_hot"])

        return single_frame_pred, all_frames_pred

    def predict_frames(self, frames: np.ndarray):
        assert len(frames.shape) == 4 and frames.shape[1:] == self._input_size, \
            "[TransNetV2] Input shape must be [frames, height, width, 3]."

        def input_iterator():
            # return windows of size 100 where the first/last 25 frames are from the previous/next batch
            # the first and last window must be padded by copies of the first and last frame of the video
            no_padded_frames_start = 25
            no_padded_frames_end = 25 + 50 - (len(frames) % 50 if len(frames) % 50 != 0 else 50)  # 25 - 74

            start_frame = np.expand_dims(frames[0], 0)
            end_frame = np.expand_dims(frames[-1], 0)
            padded_inputs = np.concatenate(
                [start_frame] * no_padded_frames_start + [frames] + [end_frame] * no_padded_frames_end, 0
            )

            ptr = 0
            while ptr + 100 <= len(padded_inputs):
                out = padded_inputs[ptr:ptr + 100]
                ptr += 50
                yield out[np.newaxis]

        predictions = []

        for inp in input_iterator():
            single_frame_pred, all_frames_pred = self.predict_raw(inp)
            predictions.append((single_frame_pred.numpy()[0, 25:75, 0],
                                all_frames_pred.numpy()[0, 25:75, 0]))

            print("\r[TransNetV2] Processing video frames {}/{}".format(
                min(len(predictions) * 50, len(frames)), len(frames)
            ), end="")
        print("")

        single_frame_pred = np.concatenate([single_ for single_, all_ in predictions])
        all_frames_pred = np.concatenate([all_ for single_, all_ in predictions])

        return single_frame_pred[:len(frames)], all_frames_pred[:len(frames)]  # remove extra padded frames

    def predict_video(self, video_fn: str):
        try:
            import ffmpeg
        except ModuleNotFoundError:
            raise ModuleNotFoundError("For `predict_video` function `ffmpeg` needs to be installed in order to extract "
                                      "individual frames from video file. Install `ffmpeg` command line tool and then "
                                      "install python wrapper by `pip install ffmpeg-python`.")

        print("[TransNetV2] Extracting frames from {}".format(video_fn))
        video_stream, err = ffmpeg.input(video_fn).output(
            "pipe:", format="rawvideo", pix_fmt="rgb24", s="48x27"
        ).run(capture_stdout=True, capture_stderr=True)

        video = np.frombuffer(video_stream, np.uint8).reshape([-1, 27, 48, 3])
        return (video, *self.predict_frames(video))

    @staticmethod
    def predictions_to_scenes(predictions: np.ndarray, threshold: float = 0.5):
        predictions = (predictions > threshold).astype(np.uint8)

        scenes = []
        t, t_prev, start = -1, 0, 0
        for i, t in enumerate(predictions):
            if t_prev == 1 and t == 0:
                start = i
            if t_prev == 0 and t == 1 and i != 0:
                scenes.append([start, i])
            t_prev = t
        if t == 0:
            scenes.append([start, i])

        # just fix if all predictions are 1
        if len(scenes) == 0:
            return np.array([[0, len(predictions) - 1]], dtype=np.int32)

        return np.array(scenes, dtype=np.int32)

    @staticmethod
    def visualize_predictions(frames: np.ndarray, predictions):
        from PIL import Image, ImageDraw

        if isinstance(predictions, np.ndarray):
            predictions = [predictions]

        ih, iw, ic = frames.shape[1:]
        width = 25

        # pad frames so that length of the video is divisible by width
        # pad frames also by len(predictions) pixels in width in order to show predictions
        pad_with = width - len(frames) % width if len(frames) % width != 0 else 0
        frames = np.pad(frames, [(0, pad_with), (0, 1), (0, len(predictions)), (0, 0)])

        predictions = [np.pad(x, (0, pad_with)) for x in predictions]
        height = len(frames) // width

        img = frames.reshape([height, width, ih + 1, iw + len(predictions), ic])
        img = np.concatenate(np.split(
            np.concatenate(np.split(img, height), axis=2)[0], width
        ), axis=2)[0, :-1]

        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)

        # iterate over all frames
        for i, pred in enumerate(zip(*predictions)):
            x, y = i % width, i // width
            x, y = x * (iw + len(predictions)) + iw, y * (ih + 1) + ih - 1

            # we can visualize multiple predictions per single frame
            for j, p in enumerate(pred):
                color = [0, 0, 0]
                color[(j + 1) % 3] = 255

                value = round(p * (ih - 1))
                if value != 0:
                    draw.line((x + j, y, x + j, y - value), fill=tuple(color), width=1)
        return img



import cv2
import numpy as np
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False
import glob
import cv2
import os

from sklearn.cluster import KMeans, spectral_clustering
from sklearn.metrics import silhouette_score, pairwise_distances_argmin_min
from scipy.signal import find_peaks

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

import tensorflow as tf
import shutil

#FUNCTIONS FOR CLUSTERING FOR KEY FRAMES

def sort_frames(list_of_frames, video_id, filename):
    return sorted(list_of_frames, key=lambda x: int(x.split(f'/content/PCA-CBVR/DB/UCF101/frames/{video_id}/{filename}/')[1].split('.jpeg')[0]))

def get_image_features(frames_filepaths):
  image_features = []
  model = VGG16(weights='imagenet', include_top=False)
  for img_path in frames_filepaths:
      print(img_path)
      img = image.load_img(img_path, target_size=(224, 224))
      img_data = image.img_to_array(img)
      img_data = np.expand_dims(img_data, axis=0)
      img_data = preprocess_input(img_data)

      vgg16_feature = model.predict(img_data)
      image_features.append(np.array(vgg16_feature).flatten())
  return image_features

def kmeans(image_features, n_clusters):
  silhouette = np.zeros(n_clusters)
  for k in range(2, n_clusters):
    kmeans = KMeans(n_clusters=k).fit_predict(image_features)
    silhouette[k] = silhouette_score(image_features, kmeans)
    print(f'{k} cluster complete')
  return silhouette

def random_frame_idxs_for_cluster(kmeans_labels):
  idxs = []
  for label in range(len(np.unique(kmeans_labels))):
    idxs.append(np.random.choice(np.argwhere(kmeans_labels == label).flatten(), 1)[0])
  return np.sort(np.array(idxs))

def closest_to_centroid_frames(kmeans, image_list, image_features):
  closest, _ = pairwise_distances_argmin_min(
      kmeans.cluster_centers_, image_features)
  return np.array(image_list)[closest]

def create_video_from_frames(frames, title, fps=30, fourcc=cv2.VideoWriter_fourcc('M','J','P','G')):
    img_array = []
    for filename in frames:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)

    out = cv2.VideoWriter(title, fourcc, 30, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()


for category in CATEGORIES:
  print("Processing category %s" % category)
  cou = 0
  # Get all files in current category's folder.
  #folder_path = os.path.join("..", "R3D50", category)
  filenames = os.listdir('/content/PCA-CBVR/DB/UCF101/frames/' + category)
  print(filenames)
  for filename in filenames:
    #VIDEO SEGMENTATION DONE USING PRETRAINED TransnetV2
    model = TransNetV2() 


    video_frames, single_frame_predictions, all_frame_predictions = model.predict_video('/content/PCA-CBVR/DB/UCF101/videos/' + category + '/'+ filename + '.avi')

    predictions = np.stack([single_frame_predictions, all_frame_predictions], 1)
    #np.savetxt(file + ".predictions.txt", predictions, fmt="%.6f")

    scenes = model.predictions_to_scenes(single_frame_predictions)
    sbd_frames = []
    dir = '/content/PCA-CBVR/DB/UCF101/frames/' + category + '/' + filename
    for i in range(0,len(scenes)):
      #print(scenes[i][1])

      sbd_frames.append(dir + "/" + str(scenes[i][0]) + ".jpeg")
      sbd_frames.append(dir + "/" + str(scenes[i][1]) + ".jpeg")


    #sbd_frames = np.array(sbd_frames)
    print(sbd_frames)

    #frames = os.listdir('/content/PCA-CBVR/DB/UCF101/frames/' + category + "/" + filename)
    #ec_images = glob.glob('/content/PCA-CBVR/DB/UCF101/frames/' + category + "/" + filename + '/*')
    ec_images = sbd_frames
    print(ec_images)
    ec_images = sort_frames(ec_images, category, filename)
    images_features = get_image_features(ec_images)
    kmeans5 = KMeans(n_clusters=2, random_state=42).fit(images_features)
    five_frames = closest_to_centroid_frames(kmeans5, ec_images, images_features)
    sorted_five_frames = sort_frames(five_frames, category, filename)
    original_frames = sort_frames(sorted_five_frames, category, filename)
    #print(original_frames)
    try: 
      if not os.path.exists(f'/content/PCA-CBVR/DB/UCF101/New/{category}'): 
        os.makedirs(f'/content/PCA-CBVR/DB/UCF101/New/{category}')
          
        
    except OSError: 
      print ('Error: Creating directory of data')
    create_video_from_frames(original_frames, '/content/PCA-CBVR/DB/UCF101/New/'+ category + '/' + filename +'.avi')


